# À propos de ce livret



Recherches liées au développement de cette méthode : 
>Yu, Hsiang-Fu, Nikhil Rao et Inderjit S. Dhillon (2016). [**Temporal regularized matrix factorization for high-dimensional time series prediction**](http://www.cs.utexas.edu/~rofuyu/papers/tr-mf-nips.pdf).


La méthode implantés provient de ces sources : 
>[**Dépôt 1**](https://github.com/xinychen/transdim) | *Version en python*



**Note sur le cachier**  


## Préparation préalable à l'utilisation

In [1]:
# Utilités
import os
import numpy as np
from numpy.linalg import inv as inv
import pandas as pd
import time

# Chargement des données
import scipy.io
import json

# Barre de progression
from tqdm.auto import trange
from tqdm import tqdm_notebook, notebook, tqdm

# Présentation des résultats
import matplotlib.pyplot as plt

In [2]:
parametres = {
    "dossier": "data/electricite",
    "dossier_experience": "exp/electricite",
    "fichier_base": "/electricite_50",
    "manquants": "10", # 50 % de manquants
    "modele": "TRMF"
             }

# Données

## Chargement des données

In [3]:
## Chargement des données
dossier = parametres["dossier"]
fichier_complet = "{:}.mat".format(parametres["fichier_base"])
fichier_binaire = "{:}_{:}.mat".format(parametres["fichier_base"],
                                       parametres["manquants"])

mat_complet = scipy.io.loadmat(dossier + fichier_complet)["mat"]
mat_binaire = scipy.io.loadmat(dossier + fichier_binaire)["mat"]
mat_manquants = mat_complet * mat_binaire

index = np.where((mat_complet != 0) & (mat_binaire == 0))

## Format des données

# Partie 2 : Modèle

In [4]:
def MAPE(mat_complet, mat_hat, index):
    mape = np.sum(
            np.abs(mat_complet[index] - mat_hat[index]) /
            mat_complet[index]) / mat_complet[index].shape[0]
    
    return mape

In [5]:
def RMSE(mat_complet, mat_hat, index):
    rmse = np.sqrt(
        np.sum((mat_complet[index] - mat_hat[index])**2) /
        mat_complet[index].shape[0])
    return rmse

## Partie 2.1 : Spécification du modèle

In [6]:
temps_sleep = 0.00000001

def TRMF(dense_mat, sparse_mat, init_para, init_hyper, time_lags, maxiter):
    """
    Temporal Regularized Matrix Factorization, TRMF.
    
    Paramètres en entrée :
        dense_mat
        sparse_mat
        init_para
        init_hyper
        time_lags
        maxiter
    
    
    Paramètres en sortie : 
        mat_hat : Matrice imputée
        mape : MAPe
        rmse : Erreur au carré
    
    
    """
    # Variables bidons pour la présentation des résultats
    lrmse = []
    lmape = []

    ## Initialize parameters
    W = init_para["W"]
    X = init_para["X"]
    theta = init_para["theta"]

    ## Set hyperparameters
    lambda_w = init_hyper["lambda_w"]
    lambda_x = init_hyper["lambda_x"]
    lambda_theta = init_hyper["lambda_theta"]
    eta = init_hyper["eta"]

    dim1, dim2 = sparse_mat.shape
    pos_train = np.where(sparse_mat != 0)
    pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    binary_mat = sparse_mat.copy()
    binary_mat[pos_train] = 1
    d, rank = theta.shape

    for it in notebook.tqdm(list(range(maxiter)),
                            desc="Itérations",
                            leave=False):
        ## Update spatial matrix W
        for i in notebook.tqdm(list(range(dim1)),
                               desc="Mise à jour W",
                               leave=False):
            pos0 = np.where(sparse_mat[i, :] != 0)
            Xt = X[pos0[0], :]
            vec0 = np.matmul(Xt.T, sparse_mat[i, pos0[0]])
            mat0 = inv(np.matmul(Xt.T, Xt) + lambda_w * np.eye(rank))
            W[i, :] = np.matmul(mat0, vec0)
        # time.sleep(temps_sleep)
        ## Update temporal matrix X
        for t in notebook.tqdm(list(range(dim2)),
                               desc="Mise à jour X",
                               leave=False):
            pos0 = np.where(sparse_mat[:, t] != 0)
            Wt = W[pos0[0], :]
            Mt = np.zeros((rank, rank))
            Nt = np.zeros(rank)
            ##
            if t < np.max(time_lags):
                Pt = np.zeros((rank, rank))
                Qt = np.zeros(rank)
            else:
                Pt = np.eye(rank)
                Qt = np.einsum('ij, ij -> j', theta, X[t - time_lags, :])
            ##
            if t < dim2 - np.min(time_lags):
                ##
                if t >= np.max(time_lags) and t < dim2 - np.max(time_lags):
                    index = list(range(0, d))
                else:
                    index = list(
                        np.where((t + time_lags >= np.max(time_lags))
                                 & (t + time_lags < dim2)))[0]
                ##
                for k in index:
                    Ak = theta[k, :]
                    Mt += np.diag(Ak**2)
                    theta0 = theta.copy()
                    theta0[k, :] = 0
                    Nt += np.multiply(
                        Ak, X[t + time_lags[k], :] -
                        np.einsum('ij, ij -> j', theta0,
                                  X[t + time_lags[k] - time_lags, :]))
#                # time.sleep(temps_sleep / 10)
            ## Vec mu
            vec0 = np.matmul(
                Wt.T, sparse_mat[pos0[0], t]) + lambda_x * Nt + lambda_x * Qt
            ## Vec lambda
            mat0 = inv(
                np.matmul(Wt.T, Wt) + lambda_x * Mt + lambda_x * Pt +
                (lambda_x * eta * np.eye(rank))) # Ajout eta qui n'était plus (?) à vérifier
            X[t, :] = np.matmul(mat0, vec0)
        # time.sleep(temps_sleep)
        ## Update AR coefficients theta
        for k in notebook.tqdm(list(range(d)),
                               desc="Mise à jour AR coefficients theta",
                               leave=False):
            theta0 = theta.copy()
            theta0[k, :] = 0
            mat0 = np.zeros((dim2 - np.max(time_lags), rank))
            for L in range(d):
                mat0 += np.matmul(
                    X[np.max(time_lags) - time_lags[L]:dim2 - time_lags[L], :],
                    np.diag(theta0[L, :]))
            VarPi = X[np.max(time_lags):dim2, :] - mat0
            var1 = np.zeros((rank, rank))
            var2 = np.zeros(rank)
            # time.sleep(temps_sleep)
            for t in range(np.max(time_lags), dim2):
                B = X[t - time_lags[k], :]
                var1 += np.diag(np.multiply(B, B))
                var2 += np.matmul(np.diag(B), VarPi[t - np.max(time_lags), :])
            theta[k, :] = np.matmul(
                inv(var1 + lambda_theta * np.eye(rank) / lambda_x), var2)
        # time.sleep(temps_sleep)

        # Matrice imputée
        mat_hat = np.matmul(W, X.T)

        # Performance du modèle
        mape = np.sum(
            np.abs(dense_mat[pos_test] - mat_hat[pos_test]) /
            dense_mat[pos_test]) / dense_mat[pos_test].shape[0]
        rmse = np.sqrt(
            np.sum((dense_mat[pos_test] - mat_hat[pos_test])**2) /
            dense_mat[pos_test].shape[0])

        # Mise en liste pour la visualisation
        lmape.append(mape)
        lrmse.append(rmse)

        if (it + 1) % 50 == 0:  #200 == 0:
            print(('Iter: {}'.format(it + 1)))
            print(('Imputation MAPE: {:}'.format(mape)))
            print(('Imputation RMSE: {:}'.format(rmse)))
            print()

        # Si le rmse ne diminue pas, arrêter l'entrainement
        if lrmse[len(lrmse) - 1] < rmse:
            rmse = lrmse[len(lrmse) - 1]  # Sauvegarder le dernier RMSE
            break
            print("Le RMSE ne diminue pas. Arrêt de l'entrainement.")

    return mat_hat, mape, rmse, lmape, lrmse, W, X

In [7]:
maxiter = 100

## Mettre le rang et les time lag dans les init
rang = 10

init_hyper = {"lambda_w": 1, 
              "lambda_x": 100, 
              "lambda_theta": 100, 
              "eta": 0.05}


time_lags = np.array([1])  # D'ordre 1
d = time_lags.shape[0]

# Formation des données en entrée
dim1, dim2 = mat_manquants.shape
np.random.seed(2020)  # Reproductibilité
W = 0.1 * np.random.rand(dim1, rang)
np.random.seed(2020)  # Reproductibilité
X = 0.1 * np.random.rand(dim2, rang)
theta = 0.1 * np.random.rand(d, rang)
init_para = {"W": W, "X": X, "theta": theta}

import time
start = time.time()
mat_hat, mape, rmse, lmape, lrmse, W, X = TRMF(mat_complet, mat_manquants, init_para,
                                         init_hyper, time_lags, maxiter)
end = time.time()



Iter: 50
Imputation MAPE: 0.05681208005448896
Imputation RMSE: 7.979413211369855



Iter: 100
Imputation MAPE: 0.05614366896748751
Imputation RMSE: 7.7425782244662456



In [8]:
temps_entrainement = np.round(end - start, 6)

## Préparation pour le rapport

### Matrice imputée

In [9]:
dossier_experience = parametres["dossier_experience"]

## Créer le dossier s'il n'existe pas
if not os.path.exists(dossier_experience):
    os.mkdir(dossier_experience)

fichier_experience = "{:}_{:}_{:}".format(parametres["fichier_base"],
                                          parametres["manquants"],
                                          parametres["modele"])
nom_fichier = "{:}{:}_r{:}-w{:}x{:}t{:}e{:}.mat".format(
    dossier_experience, fichier_experience, rang, init_hyper["lambda_w"],
    init_hyper["lambda_x"], init_hyper["lambda_theta"],
    int(init_hyper["eta"] * 100))

save_mat_hat = {"mat": mat_hat}
scipy.io.savemat(nom_fichier, save_mat_hat)

### Tableau comparatif

In [10]:
hyperparam = "Rang: {:} | Lambda W: {:} | Lambda X: {:} | Lambda theta: {:} | eta: {:} | Lags: {:}".format(
    rang, init_hyper["lambda_w"], init_hyper["lambda_x"],
    init_hyper["lambda_theta"], init_hyper["eta"], time_lags)

mape = np.round(MAPE(mat_complet, mat_hat, index), 6)
rmse = np.round(RMSE(mat_complet, mat_hat, index), 6)

param_tableau = {
    "modele":"TRMF",
    "hyperparametres":  hyperparam,
    "nom_fichier":fichier_experience,
    "rmse":rmse,
    "mape":mape,
    "temps":temps_entrainement
}
df_comparatif = pd.DataFrame(param_tableau.items()).set_index(0).T

df_comparatif.to_csv("exp/fichier_comparatif.csv", mode='a', header=False)

### Graphs : Séries

In [11]:
## Série 1
y1_pred = mat_hat[1, ].astype(np.float)
y1_orig = mat_complet[1, ].astype(np.float)
liste1 = [list(y1_pred), list(y1_orig)]
nom_fichier = "{:}{:}_r{:}-w{:}x{:}t{:}e{:}_L1.json".format(
    dossier_experience, fichier_experience, rang, init_hyper["lambda_w"],
    init_hyper["lambda_x"], init_hyper["lambda_theta"],
    int(init_hyper["eta"] * 100))
with open(nom_fichier, 'w') as outfile:
    json.dump(liste1, outfile)

In [12]:
## Série 2
y2_pred = mat_hat[2, ].astype(np.float)
y2_orig = mat_complet[2, ].astype(np.float)
liste2 = [list(y2_pred), list(y2_orig)]
nom_fichier = "{:}{:}_r{:}-w{:}x{:}t{:}e{:}_L2.json".format(
    dossier_experience, fichier_experience, rang, init_hyper["lambda_w"],
    init_hyper["lambda_x"], init_hyper["lambda_theta"],
    int(init_hyper["eta"] * 100))
with open(nom_fichier, 'w') as outfile:
    json.dump(liste2, outfile)

In [13]:
## Série 3
y3_pred = mat_hat[3, ].astype(np.float)
y3_orig = mat_complet[3, ].astype(np.float)
liste3 = [list(y3_pred), list(y3_orig)]
nom_fichier = "{:}{:}_r{:}-w{:}x{:}t{:}e{:}_L3.json".format(
    dossier_experience, fichier_experience, rang, init_hyper["lambda_w"],
    init_hyper["lambda_x"], init_hyper["lambda_theta"],
    int(init_hyper["eta"] * 100))
with open(nom_fichier, 'w') as outfile:
    json.dump(liste3, outfile)

In [14]:
## Série 4
y4_pred = mat_hat[4, ].astype(np.float)
y4_orig = mat_complet[4, ].astype(np.float)
liste4 = [list(y4_pred), list(y4_orig)]
nom_fichier = "{:}{:}_r{:}-w{:}x{:}t{:}e{:}_L4.json".format(
    dossier_experience, fichier_experience, rang, init_hyper["lambda_w"],
    init_hyper["lambda_x"], init_hyper["lambda_theta"],
    int(init_hyper["eta"] * 100))
with open(nom_fichier, 'w') as outfile:
    json.dump(liste4, outfile)

### Graph: Comparatif

In [15]:
## Préparation des données

## Capter les index des éléments imputés
x_pred = mat_hat[index]
y_original = mat_complet[index]

## Avoir la distance entre les deux éléments
dist_axe = np.abs(x_pred - y_original)

## Créer un dataframe en forme "longue"
df = pd.DataFrame([x_pred, y_original, dist_axe]).T
df.columns = ['x_pred', 'y_original', 'dist_axe']

# Déterminer des quantiles pour le graphique
q0 = df.dist_axe.quantile(0.0)
q1 = df.dist_axe.quantile(0.25)
q2 = df.dist_axe.quantile(0.5)
q3 = df.dist_axe.quantile(0.75)
q4 = df.dist_axe.quantile(1.0)

## Créer une colonne avec les catégories
df['size'] = [
    1 if q1 > x >= q0 else
    2 if q2 > x >= q1 else 3 if q3 > x >= q2 else 4 if q4 > x >= q3 else 5
    for x in df['dist_axe']
]

print((
    "Quantiles : \n\t{0:6f} \n\t{1:6f} \n\t{2:6f} \n\t{3:6f} \n\t{4:6f}"
    .format(q0, q1, q2, q3, q4)))

Quantiles : 
	0.124523 
	1.597892 
	4.196699 
	7.078513 
	19.112198


In [16]:
nom_fichier = "{:}{:}_r{:}-w{:}x{:}t{:}e{:}_comparaison.mat".format(
    dossier_experience, fichier_experience, rang, init_hyper["lambda_w"],
    init_hyper["lambda_x"], init_hyper["lambda_theta"],
    int(init_hyper["eta"] * 100))
## Mettre en numpy
df = df.to_numpy()

## Sauvegarde du fichier
graph_mat = {"index_comparaison": df}
scipy.io.savemat(nom_fichier, graph_mat)

In [17]:
print(nom_fichier)

exp/electricite/electricite_50_10_TRMF_r10-w1x100t100e5_comparaison.mat


## Entrainement du modèle